In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/heart-disease-uci/heart.csv


In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.model_selection import GridSearchCV
np.random.seed(42)

# **Reaging and show data from csv file**

In [3]:
df = pd.read_csv("../input/heart-disease-uci/heart.csv")
df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,57,0,0,140,241,0,1,123,1,0.2,1,0,3,0
299,45,1,3,110,264,0,1,132,0,1.2,1,0,3,0
300,68,1,0,144,193,1,1,141,0,3.4,1,2,3,0
301,57,1,0,130,131,0,1,115,1,1.2,1,1,3,0


In [4]:
df.columns

Index(['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach',
       'exang', 'oldpeak', 'slope', 'ca', 'thal', 'target'],
      dtype='object')

In [5]:
df["target"].unique

<bound method Series.unique of 0      1
1      1
2      1
3      1
4      1
      ..
298    0
299    0
300    0
301    0
302    0
Name: target, Length: 303, dtype: int64>

In [6]:
df["target"].value_counts()

1    165
0    138
Name: target, dtype: int64

# **handle missing data**

In [7]:
df.info()
df.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trestbps  303 non-null    int64  
 4   chol      303 non-null    int64  
 5   fbs       303 non-null    int64  
 6   restecg   303 non-null    int64  
 7   thalach   303 non-null    int64  
 8   exang     303 non-null    int64  
 9   oldpeak   303 non-null    float64
 10  slope     303 non-null    int64  
 11  ca        303 non-null    int64  
 12  thal      303 non-null    int64  
 13  target    303 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 33.3 KB


age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64

# **Training data**

In [8]:
x = df.drop("target" , axis =1)
y = df["target"]

In [9]:
x_train , x_test , y_train , y_test = train_test_split(x ,y , test_size=0.3)     # test by 30%


# **choose modle (estimator)**

In [10]:
from sklearn.ensemble import RandomForestClassifier
model=RandomForestClassifier(criterion='entropy', max_depth=7, n_estimators=70,
                       n_jobs=-1)

In [11]:
# testing
# train (input , output)
scalled = model.fit(x_train , y_train )

# validate modle 
#print(model.score(x_train , y_train))
print(model.score(x_test , y_test))

0.8241758241758241


really score for model

In [12]:
from sklearn.model_selection import cross_val_score
validator = cross_val_score(model ,x, y ,cv =5)
print(np.mean(validator)) 

0.8083060109289617


# tune modle prams

It is better to compensate with the parameters resulting from the Tuning process in the model

In [13]:
parameters ={'n_estimators':(10,30,50,70,90,100),
            'criterion':('gini' , 'entropy'),
            'max_depth':(3,5,7,9),
            'max_features':('auto' , 'sqrt'),
            'min_samples_split':(2,4,6)
}
grid = GridSearchCV(RandomForestClassifier(n_jobs=-1,oob_score=False), param_grid=parameters ,cv =3 , verbose=True)
tune_grid = grid.fit(x_train , y_train)
print(tune_grid.best_estimator_)

Fitting 3 folds for each of 288 candidates, totalling 864 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


RandomForestClassifier(criterion='entropy', max_depth=3, max_features='sqrt',
                       n_estimators=10, n_jobs=-1)


[Parallel(n_jobs=1)]: Done 864 out of 864 | elapsed:  3.4min finished


**report**

In [14]:
from sklearn.metrics import classification_report
y_predicted = model.predict(x_test)
print(classification_report(y_test , y_predicted))

              precision    recall  f1-score   support

           0       0.80      0.80      0.80        41
           1       0.84      0.84      0.84        50

    accuracy                           0.82        91
   macro avg       0.82      0.82      0.82        91
weighted avg       0.82      0.82      0.82        91

